In [7]:
!python3 -m pip install --quiet nibabel nnunetv2

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
import huggingface_hub as hh

In [3]:
hh.login(token="MY_TOKEN")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/admin/.cache/huggingface/token
Login successful


In [4]:
results_path = hh.hf_hub_download(
    repo_id="osbm/prostate158",
    filename="nnUNet_raw.zip",
    repo_type="dataset",
    local_dir=".",
)


In [6]:
!unzip nnUNet_raw.zip


Archive:  nnUNet_raw.zip
   creating: nnUNet_raw/
   creating: nnUNet_raw/Dataset001_Prostate158/
   creating: nnUNet_raw/Dataset001_Prostate158/labelsTs/
  inflating: nnUNet_raw/Dataset001_Prostate158/labelsTs/PROSTATE_17.nii.gz  
  inflating: nnUNet_raw/Dataset001_Prostate158/labelsTs/PROSTATE_14.nii.gz  
  inflating: nnUNet_raw/Dataset001_Prostate158/labelsTs/PROSTATE_1.nii.gz  
  inflating: nnUNet_raw/Dataset001_Prostate158/labelsTs/PROSTATE_15.nii.gz  
  inflating: nnUNet_raw/Dataset001_Prostate158/labelsTs/PROSTATE_16.nii.gz  
  inflating: nnUNet_raw/Dataset001_Prostate158/labelsTs/PROSTATE_13.nii.gz  
  inflating: nnUNet_raw/Dataset001_Prostate158/labelsTs/PROSTATE_19.nii.gz  
  inflating: nnUNet_raw/Dataset001_Prostate158/labelsTs/PROSTATE_6.nii.gz  
  inflating: nnUNet_raw/Dataset001_Prostate158/labelsTs/PROSTATE_2.nii.gz  
  inflating: nnUNet_raw/Dataset001_Prostate158/labelsTs/PROSTATE_18.nii.gz  
  inflating: nnUNet_raw/Dataset001_Prostate158/labelsTs/PROSTATE_5.nii.gz  
  

In [8]:
import os
os.environ['nnUNet_raw'] = 'nnUNet_raw'
os.environ['nnUNet_preprocessed'] = 'nnUNet_preprocessed'
os.environ['nnUNet_results'] = 'nnUNet_results'


In [9]:
!nnUNetv2_plan_and_preprocess -d 1

Fingerprint extraction...
Dataset001_Prostate158
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
  4%|█▌                                         | 5/139 [00:00<00:17,  7.81it/s]WARNING! Not all input images have the same direction!
Directions:
[(1.0, -4.896998000198151e-12, 4.217266257393384e-18, 4.817624131722882e-12, 0.9837915423576415, 0.1793159055164187, -8.781138086096137e-13, -0.1793159256328695, 0.9837915460242717), (1.0, -4.896998000198151e-12, 4.217266257393384e-18, 4.817624131722882e-12, 0.9837915423576415, 0.1793159055164187, -8.781138086096137e-13, -0.1793159256328695, 0.9837915460242717), (1.0, -4.896997988160078e-12, 4.2146722059781e-18, 4.817624530709464e-12, 0.9837915399392304, 0.17931592474577226, -8.781139083562592e-13, -0.17931593890114042, 0.983791542519333)]
Image files:
['nnUNet_raw/Dataset001_Prostate158/imagesTr/PROSTATE_103_0000.nii.gz', 'nnUNet_raw/Dataset001_Prostate158/imagesTr/PROSTATE_103_0001.nii.gz', 'nnUNet_raw/Data

In [11]:
!nnUNetv2_train 1 3d_fullres 0 -tr nnUNetTrainer_20epochs

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################


This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nnUNetPlans_3d_fullres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 2, 'patch_size': [28, 256, 256], 'median_image_size_in_voxels': [25.0, 270.0, 270.0], 'spacing': [2.999998998641968, 0.4017857015132904, 0.4017857015132904], 'normalization_schemes': ['ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization'], 'use_mask_for_norm': [False, False, False], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder'

In [20]:
from huggingface_hub import HfApi
api = HfApi()
repo_name = "osbm/prostate128_nnUNet_3d_fullres_20_epoch"
from huggingface_hub import create_repo
create_repo(
    repo_id=repo_name, 
    private=False,
    exist_ok=True,
    repo_type="dataset",
)
api.upload_folder(
    folder_path="nnUNet_results",
    path_in_repo="nnUNet_results",
    repo_id=repo_name,
    repo_type="dataset",
)
api.upload_folder(
    folder_path="nnUNet_preprocessed",
    repo_id=repo_name,
    path_in_repo="nnUNet_preprocessed",
    repo_type="dataset",
)
api.upload_folder(
    folder_path="outputs",
    path_in_repo="outputs",
    repo_id=repo_name,
    repo_type="dataset",
)

api.upload_folder(
    folder_path="outputs_post_processed",
    path_in_repo="outputs_post_processed",
    repo_id=repo_name,
    repo_type="dataset",
)

PROSTATE_116.nii.gz:   0%|          | 0.00/8.10k [00:00<?, ?B/s]
PROSTATE_117.nii.gz:   0%|          | 0.00/6.22k [00:00<?, ?B/s]


PROSTATE_133.nii.gz:   0%|          | 0.00/8.75k [00:00<?, ?B/s]



PROSTATE_145.nii.gz:   0%|          | 0.00/9.11k [00:00<?, ?B/s]

PROSTATE_119.nii.gz:   0%|          | 0.00/8.21k [00:00<?, ?B/s]




Upload 20 LFS files:   0%|          | 0/20 [00:00<?, ?it/s]
PROSTATE_117.nii.gz: 100%|██████████| 6.22k/6.22k [00:00<00:00, 7.82kB/s]
PROSTATE_116.nii.gz: 100%|██████████| 8.10k/8.10k [00:00<00:00, 10.0kB/s]


PROSTATE_133.nii.gz: 100%|██████████| 8.75k/8.75k [00:00<00:00, 10.6kB/s]



PROSTATE_145.nii.gz: 100%|██████████| 9.11k/9.11k [00:00<00:00, 11.0kB/s]

PROSTATE_133.nii.gz: 100%|██████████| 8.75k/8.75k [00:00<00:00, 9.59kB/s]
PROSTATE_116.nii.gz: 100%|██████████| 8.10k/8.10k [00:00<00:00, 8.43kB/s]
PROSTATE_145.nii.gz: 100%|██████████| 9.11k/9.11k [00:00<00:00, 9.64kB/s]
PROSTATE_119.nii.gz: 100%|██████████| 8.21k/8.21k [00:00<00:00, 8.25kB/s]
PROSTAT

'https://huggingface.co/datasets/osbm/prostate128_nnUNet_3d_fullres_20_epoch/tree/main/outputs_post_processed'

In [17]:
!nnUNetv2_find_best_configuration 1 -c "3d_fullres" --disable_ensembling -tr nnUNetTrainer_20epochs -f 0


***All results:***
nnUNetTrainer_20epochs__nnUNetPlans__3d_fullres: 0.5937358543720834

*Best*: nnUNetTrainer_20epochs__nnUNetPlans__3d_fullres: 0.5937358543720834

***Determining postprocessing for best model/ensemble***
Removing all but the largest foreground region did not improve results!
Results were improved by removing all but the largest component for 1. Dice before: 0.84286 after: 0.84286
Removing all but the largest component for 2 did not improve results! Dice before: 0.679 after: 0.67829
Results were improved by removing all but the largest component for 3. Dice before: 0.25935 after: 0.26226

***Run inference like this:***

nnUNetv2_predict -d Dataset001_Prostate158 -i INPUT_FOLDER -o OUTPUT_FOLDER -f  0 -tr nnUNetTrainer_20epochs -c 3d_fullres -p nnUNetPlans

***Once inference is completed, run postprocessing like this:***

nnUNetv2_apply_postprocessing -i OUTPUT_FOLDER -o OUTPUT_FOLDER_PP -pp_pkl_file nnUNet_results/Dataset001_Prostate158/nnUNetTrainer_20epochs__nnUNetP

In [18]:
!nnUNetv2_predict -i "nnUNet_raw/Dataset001_Prostate158/imagesTs" -o "outputs" -d 1 -c "3d_fullres" --save_probabilities -f 0 -tr nnUNetTrainer_20epochs


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 19 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 19 cases that I would like to predict
using pin_memory on device 0
WARNING! Not all input images have the same direction!
Directions:
[(1.0, -4.897000249694329e-12, -1.9868948322662279e-16, 4.665122917598315e-12, 0.9526614854740363, -0.30403305980268575, 1.4890392160034811e-12, 0.3040330477010723, 0.9526614816119189), (1.0, -4.897000249694329e-12, -1.9868948322662279e-16, 4.665122917598315e-12, 0.9526614854740363, -0.30403305980268575, 1.4890392160034811e-12, 0.30403304

In [19]:
!nnUNetv2_apply_postprocessing -i outputs -o outputs_post_processed -pp_pkl_file nnUNet_results/Dataset001_Prostate158/nnUNetTrainer_20epochs__nnUNetPlans__3d_fullres/crossval_results_folds_0/postprocessing.pkl -np 8 -plans_json nnUNet_results/Dataset001_Prostate158/nnUNetTrainer_20epochs__nnUNetPlans__3d_fullres/crossval_results_folds_0/plans.json